# Wrangle Tables into Relational Model

Tables:

- Wars
- War_Deaths
- War_LargerConflicts
- War_Locations
- Locations
- Participants
- War_Participants
- Participant_ParticipantTypes
- ParticipantTypes

In [1]:
import pandas as pd

In [2]:
war_par = pd.read_csv("Data/wars_participants.csv")
war = pd.read_csv("Data/wars.csv")
orgs = pd.read_csv("Data/participants_armedorgs.csv")
states = pd.read_csv("Data/participants_states.csv")
locs = pd.read_csv("Data/war_locations.csv")

## Create Wars table

Attributes:

- warID
- warName
- startDate
- endDate
- warDescription
- prevWarID
- nextWarID

In [13]:
war_table = war[['warID', 'eventLabel', 'startDate', 'endDate', 'eventDescription', 
                 'prevWarID', 'nextWarID']].copy().drop_duplicates() \
            .rename(columns={'eventLabel':'warName', 'eventDescription':'warDescription'})

In [15]:
war_table[war_table.duplicated(subset=['warID'], keep=False)]

,warID,eventLabel,startDate,endDate,eventDescription,prevWarID,nextWarID
84,Q877450,Laotian Civil War,1953-11-09T00:00:00Z,1975-12-02T00:00:00Z,1959–1975 civil war in Laos,NaN,NaN
94,Q877450,Laotian Civil War,1959-05-23T00:00:00Z,1975-12-02T00:00:00Z,1959–1975 civil war in Laos,NaN,NaN
243,Q1773984,Batalla del pan dulce,1950-01-01T00:00:00Z,1951-01-01T00:00:00Z,Batalla entre pacistas y ateos,NaN,NaN
267,Q1773984,Batalla del pan dulce,1950-10-06T00:00:00Z,1951-01-01T00:00:00Z,Batalla entre pacistas y ateos,NaN,NaN
273,Q1773984,Batalla del pan dulce,1950-01-01T00:00:00Z,1950-10-19T00:00:00Z,Batalla entre pacistas y ateos,NaN,NaN
308,Q1773984,Batalla del pan dulce,1950-10-06T00:00:00Z,1950-10-19T00:00:00Z,Batalla entre pacistas y ateos,NaN,NaN


- for Q1773984: drop observations with less specificity (243, 267, 273) in favor of observation with most specificity (308)
- for Q877450: drop 84 based on Wikipedia entry for war, which specifies start date of 23 May 1959

In [16]:
war_table = war_table.drop([84, 243, 267, 273])

In [24]:
war_table.to_csv('Data/Wrangled/war.csv', index=False)

## Create the War tables for multivalued attributes

- War_Deaths
- War_LargerConflicts
- War_Locations

In [26]:
war_deaths = war[['warID', 'casualties']].copy().drop_duplicates().dropna()
war_deaths.to_csv('Data/Wrangled/war_deaths.csv', index=False)

In [49]:
war_locations = war[['warID', 'locationID']].copy().drop_duplicates().dropna()
war_locations.to_csv('Data/Wrangled/war_locations.csv', index=False)

In [30]:
war_largerconflict = war[['warID', 'largerConflictID']].copy().drop_duplicates().dropna()
war_largerconflict.to_csv('Data/Wrangled/war_largerconflicts.csv', index=False)

### Houston, we have a problem

In [43]:
conflictIDs = set(war['largerConflictID'].dropna().unique())
warIDs = set(war['warID'].dropna().unique())
not_wars = conflictIDs - warIDs
len(conflictIDs), len(not_wars)

(75, 41)

Out of 75 larger conflicts, 41 are not wars, and thus will not be present in the wars table, so there cannot be a foreign key to the wars table as is.

In [42]:
pwIDs = set(war['prevWarID'].dropna().unique())
warIDs = set(war['warID'].dropna().unique())
not_wars_pw = pwIDs - warIDs
len(pwIDs), len(not_wars_pw)

(8, 3)

In [46]:
nwIDs = set(war['nextWarID'].dropna().unique())
warIDs = set(war['warID'].dropna().unique())
not_wars_nw = nwIDs - warIDs
len(nwIDs), len(not_wars_nw)

(7, 2)

In [48]:
not_wars_pw, not_wars_nw

({'Q2405009', 'Q2629782', 'Q2992527'}, {'Q2405009', 'Q3446062'})

There's also a problem with some of the "next wars" and "previous wars" - 4 of them aren't in the original war list.

- Q2405009 lacks an "instance of" attribute
- Q2629782 and Q2992527 happen before 1945
- Q3446062 is an instance of conflict, not of war

## Create Locations table

- locationID
- locationName
- locationType

In [64]:
locations = locs[['locationID', 'locationLabel', 'locationDescription']].copy() \
            .rename(columns={'locationLabel':'locationName', 'locationCountryLabel': 'countryName'}) \
            .drop_duplicates()
locations.to_csv('Data/Wrangled/locations.csv', index=False)

In [66]:
locIDl = set(locations['locationID'].dropna().unique())
locIDw = set(war_locations['locationID'].dropna().unique())
locIDmissing = locIDw - locIDl
len(locIDmissing)

0

## Create War_Participants table

- warID
- participantID

In [81]:
war_participants = war_par[['warID', 'participantID']].copy().drop_duplicates().dropna()
war_participants.to_csv('Data/Wrangled/war_participants.csv', index=False)

## Create Participants table

- 

In [68]:
all_par = war_par[['participantID', 'participantLabel']].copy().drop_duplicates().dropna()

In [69]:
all_par

,participantID,participantLabel
0,Q384535,African Union – United Nations Hybrid Operatio...
1,Q801,Israel
2,Q241,Cuba
3,Q810,Jordan
4,Q1036,Uganda
...,...,...
834,Q813,Kyrgyzstan
835,Q935288,Libyan armed forces
837,Q309474,Al-Aqsa Martyrs' Brigades
841,Q218,Romania


In [72]:
orgs_par = orgs[['participantID', 'participantLabel', 'participantDescription', 'inception', 'dissolved']].copy().drop_duplicates()

In [73]:
orgs_par

,participantID,participantLabel,participantDescription,inception,dissolved
0,Q80312,Liberation Tigers of Tamil Eelam,militant organization in Sri Lanka,1976-01-01T00:00:00Z,2009-01-01T00:00:00Z
2,Q58967,Israel Defense Forces,combined military forces of Israel,1948-05-26T00:00:00Z,NaN
3,Q160805,United Nations Interim Force in Lebanon,NaN,1978-01-01T00:00:00Z,NaN
4,Q7184,NATO,intergovernmental military alliance of Western...,1949-04-04T00:00:00Z,NaN
6,Q11223,United States Air Force,air warfare branch of the United States Armed ...,1947-01-01T00:00:00Z,NaN
...,...,...,...,...,...
109,Q1674170,Turkistan Islamic Party,NaN,1997-01-01T00:00:00Z,NaN
111,Q1967703,National Resistance Army,military wing of the National Resistance Movem...,1981-06-01T00:00:00Z,1995-01-01T00:00:00Z
112,Q935288,Libyan armed forces,combined military forces of Libya,1977-01-01T00:00:00Z,NaN
113,Q1639447,Pakistan Air Force,air warfare branch of Pakistan's armed forces,1947-08-14T00:00:00Z,NaN


In [74]:
states_par = states[['participantID', 'participantLabel', 'participantDescription', 'inception', 'dissolved', 'countryCode']].copy().drop_duplicates()

In [75]:
states

,participant,participantLabel,participantDescription,countryCode,inception,dissolved,government,governmentLabel,governmentDescription,participantID,govtID
0,http://www.wikidata.org/entity/Q114,Kenya,sovereign state in East Africa,KEN,1963-01-01T00:00:00Z,NaN,NaN,NaN,NaN,Q114,NaN
1,http://www.wikidata.org/entity/Q142,France,sovereign state with mainland in Europe and se...,FRA,NaN,NaN,http://www.wikidata.org/entity/Q49890,semi-presidential system,system of government,Q142,Q49890
2,http://www.wikidata.org/entity/Q36,Poland,sovereign state in Central Europe,POL,1918-11-11T00:00:00Z,NaN,http://www.wikidata.org/entity/Q166747,parliamentary system,form of government,Q36,Q166747
3,http://www.wikidata.org/entity/Q142,France,sovereign state with mainland in Europe and se...,FRA,NaN,NaN,http://www.wikidata.org/entity/Q188961,decentralization,process of redistributing or dispersing functi...,Q142,Q188961
4,http://www.wikidata.org/entity/Q115,Ethiopia,Country in East Africa,ETH,1941-05-05T00:00:00Z,NaN,http://www.wikidata.org/entity/Q512187,federal republic,federation of states or territories with a rep...,Q115,Q512187
...,...,...,...,...,...,...,...,...,...,...,...
156,http://www.wikidata.org/entity/Q224,Croatia,sovereign state in Central Europe,HRV,1991-06-25T00:00:00Z,NaN,http://www.wikidata.org/entity/Q7270,republic,form of government where people elect leaders ...,Q224,Q7270
157,http://www.wikidata.org/entity/Q419,Peru,sovereign state in South America,PER,1821-07-28T00:00:00Z,NaN,http://www.wikidata.org/entity/Q7270,republic,form of government where people elect leaders ...,Q419,Q7270
158,http://www.wikidata.org/entity/Q971,Republic of the Congo,sovereign state in Central Africa,COG,1960-01-01T00:00:00Z,NaN,NaN,NaN,NaN,Q971,NaN
159,http://www.wikidata.org/entity/Q774,Guatemala,sovereign state in Central America,GTM,1821-01-01T00:00:00Z,NaN,NaN,NaN,NaN,Q774,NaN


In [76]:
all_parIDs = set(all_par['participantID'].dropna().unique())
state_parIDs = set(states_par['participantID'].dropna().unique())
org_parIDs = set(orgs_par['participantID'].dropna().unique())
missing_parIDs = all_parIDs - state_parIDs - org_parIDs

In [78]:
all_par[all_par['participantID'].isin(list(missing_parIDs))]

,participantID,participantLabel
0,Q384535,African Union – United Nations Hybrid Operatio...
6,Q21476759,National Forces of Liberation
9,Q130216,Nagorno-Karabakh Autonomous Oblast
10,Q244165,Artsakh
11,Q318657,Armenian Revolutionary Federation
...,...,...
819,Q18651204,Resolute Support Mission
823,Q628461,Popular Resistance Committees
831,Q818618,Democratic Front for the Liberation of Palestine
837,Q309474,Al-Aqsa Martyrs' Brigades


To-do: determine better categories to fit in all of these other participants

## Create Participant_ParticipantTypes table

- participantID
- participantTypeID

## Create ParticipantTypes table

In [3]:
states_types = states[['govtID','governmentLabel','governmentDescription']].copy().drop_duplicates().dropna(subset=['govtID'])
states_types['category'] = 'state'
states_types = states_types.rename(columns={'govtID':'typeID','governmentLabel':'typeLabel','governmentDescription':'typeDescription'})

In [4]:
orgs_types = orgs[['typeID','typeLabel','typeDescription']].copy().drop_duplicates().dropna(subset=['typeID'])
orgs_types['category'] = 'armed organization'

In [5]:
part_types = pd.concat([states_types, orgs_types])

In [6]:
part_types

,typeID,typeLabel,typeDescription,category
0,Q41614,constitutional monarchy,type of monarchy in which power is restricted ...,state
1,Q7270,republic,form of government where people elect leaders ...,state
2,Q3330103,parliamentary monarchy,form of government,state
5,Q179164,unitary state,state governed as a single unit with a supreme...,state
6,Q512187,federal republic,federation of states or territories with a rep...,state
7,Q166747,parliamentary system,form of government,state
13,Q49892,presidential system,form of government,state
14,Q49890,semi-presidential system,system of government,state
15,Q188961,decentralization,process of redistributing or dispersing functi...,state
18,Q188759,representative democracy,democracy where citizens elect a small set of ...,state
